# Summarizing Data Post QC

Author: Zan Koenig

## Index
1. [Set Default Paths](#1.-Set-Default-Paths)
2. [Read in Pre-QC Dataset and Apply Quality Control Filters](#2.-Read-in-Pre-QC-Dataset-and-Apply-Quality-Control-Filters)
3. [Organize the Dataset](#3.-Organize-the-Dataset)
4. [Annotate Table with Relatedness Information](#4.-Annotate-Table-with-Relatedness-Information)
5. [Calculate Statistics per Population](#5.-Calculate-Statistics-per-Population)
6. [Format Table for Exporting](#6.-Format-Table-for-Exporting)
7. [Export Final Table](#7.-Export-Final-Table)

# General Overview:

The purpose of this script is to format and write out a tsv which will be used to create plots and summaries of the post-QC dataset in R.

**This script contains information on how to:**
- Select specific columns from a matrix table (shortened as mt)
- Annotate filter flags onto a matrix table
- Join the columns of two matrix tables
- Join two tables
- Group a matrix table by region, population 
- Use `hl.agg.stats` to calculate statics for a metric within a population
- Count the number of samples where a filter flag is `True`


In [1]:
import hail as hl

# Functions from gnomAD library to apply genotype filters   
from gnomad.utils.filtering import filter_to_adj

In [2]:
# Initializing Hail 
hl.init()

Running on Apache Spark version 3.1.3
SparkUI available at http://mty-m.c.diverse-pop-seq-ref.internal:36703
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.109-b71b065e4bb6
LOGGING: writing to /home/hail/hail-20230317-1455-0.2.109-b71b065e4bb6.log


In [1]:
# Allow output scrolling in Jupyter nb viewer for cells with long outputs 

from IPython.core.display import HTML
css = open('format.css').read()
HTML('<style>{}</style>'.format(css))

# 1. Set Default Paths

These default paths can be edited by users as needed. It is recommended to run these tutorials without writing out datasets.

**By default, all of the dataset write out sections are shown as markdown cells. If you would like to write out your own dataset, you can copy the code and paste it into a new code cell. Don't forget to change the paths in the following cell accordingly.** 

[Back to Index](#Index)

In [3]:
# Path for HGDP+1kGP dataset prior to applying gnomAD QC filters
pre_qc_path = 'gs://gcp-public-data--gnomad/release/3.1.2/mt/genomes/gnomad.genomes.v3.1.2.hgdp_1kg_subset_dense.mt'

# PCA outliers file 
outliers_path = 'gs://gcp-public-data--gnomad/release/3.1/secondary_analyses/hgdp_1kg/pca/pca_outliers.txt'

# Paths to related and unrelated Matrix Tables (without outliers) written out in Notebook 2: PCA and Ancestry Analyses
unrelateds_path = 'gs://gcp-public-data--gnomad/release/3.1/secondary_analyses/hgdp_1kg/pca_results/unrelateds_without_outliers.mt'
relateds_path = 'gs://gcp-public-data--gnomad/release/3.1/secondary_analyses/hgdp_1kg/pca_results/relateds_without_outliers.mt'

# Path for final output table in tsv format
final_table_path = 'gs://gcp-public-data--gnomad/release/3.1/secondary_analyses/hgdp_1kg/metadata_and_qc/post_qc_summary.tsv'

# 2. Read in Pre-QC Dataset and Apply Quality Control Filters

Since the post-QC mt was not written out, we run the same function as tutorial notebook 1 and 2 to apply the quality control filters to the pre-QC dataset.

**To avoid errors, make sure to run the next two cells before running any code that includes the post-QC dataset.**

**If running the cell below results in an error, double check that you used the  `--packages gnomad` argument when starting your cluster.**  

- See the tutorials [README](https://github.com/atgu/hgdp_tgp/tree/master/tutorials#readme) for more information on how to start a cluster.

<br>
<details><summary> For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 
<ul>
<li><a href="https://hail.is/docs/0.2/methods/impex.html#hail.methods.read_matrix_table"> More on  <i> read_matrix_table() </i></a></li>

<li><a href="https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.count"> More on  <i> count() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/linalg/hail.linalg.BlockMatrix.html#hail.linalg.BlockMatrix.filter_cols"> More on  <i> filter_cols() </i></a></li>

<li><a href="https://hail.is/docs/0.2/linalg/hail.linalg.BlockMatrix.html#hail.linalg.BlockMatrix.filter_rows"> More on  <i> filter_rows() </i></a></li>
</ul>
</details>

[Back to Index](#Index)

In [8]:
# Set up function to apply gnomAD's sample, variant and genotype QC filters

def run_qc(mt):
    
    ## Apply sample QC filters to dataset 
    # This filters to only samples that passed gnomAD's sample QC hard filters  
    mt = mt.filter_cols(~mt.gnomad_sample_filters.hard_filtered) # removed 31 samples
    
    ## Apply variant QC filters to dataset
    # This subsets to only PASS variants - those which passed gnomAD's variant QC
    # PASS variants have an entry in the filters field 
    mt = mt.filter_rows(hl.len(mt.filters) != 0, keep=False)

    ## Apply genotype QC filters to the dataset
    # This is done using a function imported from gnomAD and is the last step in the QC process
    mt = filter_to_adj(mt)

    return mt

In [9]:
# Read in the HGDP+1kGP pre-QC mt
pre_qc_mt = hl.read_matrix_table(pre_qc_path)

# Run QC 
mt = run_qc(pre_qc_mt)

# 3. Organize the Dataset

Here we are walking through some steps to get the post-QC dataset ready for downstream analyses. We first create a table with only sample data and then select only the columns we need for the table we will write out. Next, we create a checkpoint of that table to speed up downstream analyses.

<br>
<details><summary>For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 
<br>
<li><a href="https://hail.is/docs/0.2/utils/index.html#hail.utils.hadoop_open"> More on  <i> hadoop_open() </i></a></li>

<li><a href="https://hail.is/docs/0.2/methods/genetics.html#hail.methods.sample_qc"> More on  <i> sample_qc() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.cols"> More on  <i> cols() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.select"> More on  <i> select() </i></a></li>    
    
<li><a href="https://hail.is/docs/0.2/methods/impex.html#hail.methods.read_table"> More on  <i> read_table() </i></a></li>

<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.count"> More on  <i> count() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.checkpoint"> More on  <i> checkpoint() </i></a></li>
     
</details>

[Back to Index](#Index)

In [10]:
# Remove PCA outliers from the dataset

# Use hl.hadoop_open to read in the PCA outliers file into Hail from Google Cloud Storage
with hl.utils.hadoop_open(outliers_path) as file:
    outliers = [line.rstrip('\n') for line in file]

# Use hl.literal to convert the outliers list from a python object to a Hail expression so that it can be used to filter out samples
outliers_list = hl.literal(outliers)

# Keep the samples which are not contained in the pca outlier list
mt_without_outliers = mt.filter_cols(~outliers_list.contains(mt['s']))

mt_without_outliers.count() 

(159795273, 4096)

In [11]:
# Grab the column fields of the Matrix Table 
mt_col_table = mt_without_outliers.cols()

# Write a col table with only the columns needed for table 1
mt_col_table = mt_col_table.select(mt_col_table.hgdp_tgp_meta.genetic_region,
                             mt_col_table.hgdp_tgp_meta.population,
                             mt_col_table.sample_qc.n_snp, 
                             mt_col_table.bam_metrics.mean_coverage)

# Validity check - there should be 4096 samples 
mt_col_table.count()

2023-03-03 19:55:35.224 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


4096

# 4. Annotate Table with Relatedness Information

Relatedness information is added to the dataset so we can filter out related individuals. 

<br>
<details><summary>For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 
    
<br>
<li><a href="https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.annotate_cols"> More on  <i> annotate_cols() </i></a></li>

<li><a href="https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.counter"> More on  <i> counter() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.union_cols"> More on  <i> union_cols() </i></a></li>
    
</details>

[Back to Index](#Index)

In [44]:
# Need to get number of unrelateds annotated to the table

# Read in the unrelated and related matrix tables which were written out in Notebook 2: PCA and Ancestry Analyses
unrelateds = hl.read_matrix_table(unrelateds_path)
relateds = hl.read_matrix_table(relateds_path)

# Annotate both the unrelated and the related tables with a flag named 'unrelated' 
# Set the unrelated flag to 'True' for those in the unrelated dataset, and 'False' for those in the related dataset
unrelateds = unrelateds.annotate_cols(unrelated=True)
relateds = relateds.annotate_cols(unrelated=False)

# Use hl.cols() to obtain two tables with only the columns from the unrelated and related mts
unrelateds_cols = unrelateds.cols()
relateds_cols = relateds.cols()

# Validity check 
print(unrelateds_cols.count(), relateds_cols.count()) # 3378 unrelated and 718 related samples = 4096 total samples 

3378 718


In [45]:
# Annotate the unrelated and related mts with counts per population
unrelateds_count = unrelateds_cols.aggregate(hl.agg.counter(unrelateds_cols.hgdp_tgp_meta.population))
relateds_count = relateds_cols.aggregate(hl.agg.counter(relateds_cols.hgdp_tgp_meta.population))

# Validity check - print out the number of unrelated and related individuals per population 
print(f"Number of unrelated individuals per population: \
{unrelateds_count}\n\nNumber of related individuals per population: {relateds_count}")

Number of unrelated individuals per population: frozendict({'ACB': 94, 'ASW': 52, 'Adygei': 17, 'BEB': 99, 'Balochi': 23, 'BantuKenya': 10, 'BantuSouthAfrica': 8, 'Basque': 23, 'Bedouin': 42, 'BiakaPygmy': 22, 'Brahui': 23, 'Burusho': 24, 'CDX': 88, 'CEU': 120, 'CHB': 103, 'CHS': 103, 'CLM': 94, 'Cambodian': 9, 'Colombian': 3, 'Dai': 9, 'Daur': 9, 'Druze': 35, 'ESN': 103, 'FIN': 98, 'French': 27, 'GBR': 87, 'GIH': 100, 'GWD': 116, 'Han': 43, 'Hazara': 16, 'Hezhen': 8, 'IBS': 104, 'ITU': 102, 'Italian': 11, 'JPT': 102, 'Japanese': 29, 'KHV': 101, 'Kalash': 21, 'Karitiana': 10, 'LWK': 91, 'Lahu': 5, 'MSL': 83, 'MXL': 63, 'Makrani': 22, 'Mandenka': 20, 'Maya': 19, 'MbutiPygmy': 12, 'Melanesian': 11, 'Miao': 10, 'Mongola': 10, 'Mozabite': 25, 'Naxi': 8, 'Orcadian': 14, 'Oroqen': 8, 'PEL': 85, 'PJL': 97, 'PUR': 104, 'Palestinian': 38, 'Papuan': 17, 'Pathan': 24, 'Pima': 11, 'Russian': 25, 'STU': 98, 'San': 6, 'Sardinian': 27, 'She': 9, 'Sindhi': 22, 'Surui': 7, 'TSI': 103, 'Tu': 10, 'Tujia'

In [46]:
# Join the columns of the unrelated and related datasets
mt_combined = unrelateds.union_cols(relateds)

# Validity check - count the number of unrelateds (True values) in the mt to make sure it is as expected
print(mt_combined.aggregate_cols(hl.agg.counter(mt_combined.unrelated))) # 3378 True and 718 False

# Create a table with only the columns from the mt containing related information
# This is done since the final output will be a tsv and thus must be in table format
# Being a table of columns allows it to be annotated onto the existing mt_col_table as shown below
mt_combined_col_table = mt_combined.cols()

# Annotate the relatedness information onto the column table
mt_col_table = mt_col_table.annotate(unrelated = mt_combined_col_table[mt_col_table.s].unrelated)

frozendict({False: 718, True: 3378})


In [47]:
mt_col_table.show(5)

2022-11-18 21:04:21 Hail: INFO: Coerced sorted dataset
2022-11-18 21:04:21 Hail: INFO: Coerced sorted dataset


,,,,,
s,genetic_region,population,n_snp,mean_coverage,unrelated
str,str,str,int64,float64,bool
"""HG00096""","""EUR""","""GBR""",2566022,3.29e+01,True
"""HG00097""","""EUR""","""GBR""",2569966,3.15e+01,True
"""HG00099""","""EUR""","""GBR""",2567942,3.64e+01,True
"""HG00100""","""EUR""","""GBR""",2576696,3.02e+01,True
"""HG00101""","""EUR""","""GBR""",2565175,3.28e+01,True


# 5. Calculate Statistics per Population

In this section, we will be using `hl.agg.stats()` which calculates the following metrics for a given expression:
- min
- max
- mean
- standard deviation
- number of non-missing records
- sum
 
Using `hl.group_by()`, we calculate these statistics for each of the 78 populations in this dataset.
We also count the number of related samples within each populations by using `hl.agg.count_where()` and count the number of times the field denoting if samples are related or not is True.

<br>
<details><summary>For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 
<br>
<li><a href="https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.stats"> More on  <i> stats() </i></a></li>

<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.group_by"> More on  <i> group_by() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.count_where"> More on  <i> count_where() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.expr.TupleExpression.html#hail.expr.TupleExpression.show"> More on <i>show()</i></a></li>
     
</details>

[Back to Index](#Index)

In [49]:
# Calculate stats per population for each metric grouped by genetic region and population
table = n_snp = mt_col_table.group_by(
    mt_col_table.genetic_region, mt_col_table.population).aggregate(
    n_snp_stats = hl.agg.stats(mt_col_table.n_snp),
    cov_stats = hl.agg.stats(mt_col_table.mean_coverage),
    n_unrelated = hl.agg.count_where(mt_col_table.unrelated))

In [52]:
# Check that each of the table fields contain what we'd expect
table.show()

2022-11-18 21:08:46 Hail: INFO: Coerced sorted dataset
2022-11-18 21:08:46 Hail: INFO: Coerced sorted dataset
2022-11-18 21:08:47 Hail: INFO: Ordering unsorted dataset with network shuffle


+----------------+--------------------+------------------+-------------------+
| genetic_region | population         | n_snp_stats.mean | n_snp_stats.stdev |
+----------------+--------------------+------------------+-------------------+
| str            | str                |          float64 |           float64 |
+----------------+--------------------+------------------+-------------------+
| "AFR"          | "ACB"              |         3.14e+06 |          4.63e+04 |
| "AFR"          | "ASW"              |         3.08e+06 |          6.23e+04 |
| "AFR"          | "BantuKenya"       |         3.18e+06 |          2.01e+04 |
| "AFR"          | "BantuSouthAfrica" |         3.27e+06 |          4.15e+04 |
| "AFR"          | "BiakaPygmy"       |         3.41e+06 |          9.15e+03 |
| "AFR"          | "ESN"              |         3.20e+06 |          8.46e+03 |
| "AFR"          | "GWD"              |         3.19e+06 |          1.19e+04 |
| "AFR"          | "LWK"              |         3.19e+06 |          9.91e+03 |
| "AFR"          | "MSL"              |         3.23e+06 |          8.52e+03 |
| "AFR"          | "Mandenka"         |         3.18e+06 |          9.50e+03 |
+----------------+--------------------+------------------+-------------------+

+-----------------+-----------------+---------------+-----------------+
| n_snp_stats.min | n_snp_stats.max | n_snp_stats.n | n_snp_stats.sum |
+-----------------+-----------------+---------------+-----------------+
|         float64 |         float64 |         int64 |         float64 |
+-----------------+-----------------+---------------+-----------------+
|        2.92e+06 |        3.20e+06 |           114 |        3.58e+08 |
|        2.88e+06 |        3.18e+06 |            71 |        2.18e+08 |
|        3.14e+06 |        3.21e+06 |            12 |        3.82e+07 |
|        3.21e+06 |        3.35e+06 |             8 |        2.62e+07 |
|        3.39e+06 |        3.43e+06 |            26 |        8.86e+07 |
|        3.18e+06 |        3.23e+06 |           148 |        4.74e+08 |
|        3.15e+06 |        3.22e+06 |           176 |        5.61e+08 |
|        3.15e+06 |        3.22e+06 |            97 |        3.10e+08 |
|        3.20e+06 |        3.26e+06 |            98 |        3.17e+08 |
|        3.16e+06 |        3.21e+06 |            23 |        7.32e+07 |
+-----------------+-----------------+---------------+-----------------+

+----------------+-----------------+---------------+---------------+
| cov_stats.mean | cov_stats.stdev | cov_stats.min | cov_stats.max |
+----------------+-----------------+---------------+---------------+
|        float64 |         float64 |       float64 |       float64 |
+----------------+-----------------+---------------+---------------+
|       3.17e+01 |        2.48e+00 |      2.84e+01 |      4.20e+01 |
|       3.23e+01 |        3.36e+00 |      2.75e+01 |      5.21e+01 |
|       3.19e+01 |        3.88e+00 |      2.91e+01 |      4.40e+01 |
|       3.89e+01 |        1.03e+01 |      3.05e+01 |      6.45e+01 |
|       3.23e+01 |        3.05e+00 |      2.71e+01 |      4.01e+01 |
|       3.23e+01 |        3.23e+00 |      2.80e+01 |      5.35e+01 |
|       3.25e+01 |        2.71e+00 |      2.82e+01 |      4.34e+01 |
|       3.29e+01 |        4.32e+00 |      2.79e+01 |      6.02e+01 |
|       3.18e+01 |        2.69e+00 |      2.75e+01 |      4.56e+01 |
|       3.24e+01 |        2.65e+00 |      2.73e+01 |      4.02e+01 |
+----------------+-----------------+---------------+---------------+

+-------------+---------------+-------------+
| cov_stats.n | cov_stats.sum | n_unrelated |
+-------------+---------------+-------------+
|       int64 |       float64 |       int64 |
+-------------+---------------+-------------+
|         114 |      3.62e+03 |          94 |
|          71 |      2.29e+03 |          52 |
|          12 |      3.83e+02 |          10 |
|           8 |      3.11e+02 |           8 |
|          26 |      8.41e+02 |          22 |
|         148 |  

# 6. Format Table for Exporting

In this section, we format the table before exporting so it is in a usable format once written out. Specifically, we are flattening the table. This is done so that when the table is written out, the data is easier to work with. If the tables were written out without flattening them, the new annotated information would be in a nested structure which would make it difficult to work with outside of Hail. 

<br>
<details><summary>For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 
    
<br>
<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.flatten"> More on  <i> flatten() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.key_by"> More on <i>key_by()</i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.describe"> More on <i>describe()</i></a></li>
    
</details>

[Back to Index](#Index)

In [53]:
# Flatten out the structs created from annotating the tables
table = table.flatten()

# Change the keys of the table so that it is keyed by genetic region and population
table = table.key_by(table.genetic_region, table.population)

In [54]:
# Check format of the flattened table
table.show()

2022-11-18 21:09:47 Hail: INFO: Coerced sorted dataset
2022-11-18 21:09:47 Hail: INFO: Coerced sorted dataset
2022-11-18 21:09:48 Hail: INFO: Ordering unsorted dataset with network shuffle


,,,,,,,,,,,,,,
genetic_region,population,n_snp_stats.mean,n_snp_stats.stdev,n_snp_stats.min,n_snp_stats.max,n_snp_stats.n,n_snp_stats.sum,cov_stats.mean,cov_stats.stdev,cov_stats.min,cov_stats.max,cov_stats.n,cov_stats.sum,n_unrelated
str,str,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,int64,float64,int64
"""AFR""","""ACB""",3.14e+06,4.63e+04,2.92e+06,3.20e+06,114,3.58e+08,3.17e+01,2.48e+00,2.84e+01,4.20e+01,114,3.62e+03,94
"""AFR""","""ASW""",3.08e+06,6.23e+04,2.88e+06,3.18e+06,71,2.18e+08,3.23e+01,3.36e+00,2.75e+01,5.21e+01,71,2.29e+03,52
"""AFR""","""BantuKenya""",3.18e+06,2.01e+04,3.14e+06,3.21e+06,12,3.82e+07,3.19e+01,3.88e+00,2.91e+01,4.40e+01,12,3.83e+02,10
"""AFR""","""BantuSouthAfrica""",3.27e+06,4.15e+04,3.21e+06,3.35e+06,8,2.62e+07,3.89e+01,1.03e+01,3.05e+01,6.45e+01,8,3.11e+02,8
"""AFR""","""BiakaPygmy""",3.41e+06,9.15e+03,3.39e+06,3.43e+06,26,8.86e+07,3.23e+01,3.05e+00,2.71e+01,4.01e+01,26,8.41e+02,22
"""AFR""","""ESN""",3.20e+06,8.46e+03,3.18e+06,3.23e+06,148,4.74e+08,3.23e+01,3.23e+00,2.80e+01,5.35e+01,148,4.78e+03,103
"""AFR""","""GWD""",3.19e+06,1.19e+04,3.15e+06,3.22e+06,176,5.61e+08,3.25e+01,2.71e+00,2.82e+01,4.34e+01,176,5.72e+03,116
"""AFR""","""LWK""",3.19e+06,9.91e+03,3.15e+06,3.22e+06,97,3.10e+08,3.29e+01,4.32e+00,2.79e+01,6.02e+01,97,3.19e+03,91


In [55]:
# Check the description of the table after flattening to make sure it is what we'd expect
table.describe()

----------------------------------------
Global fields:
    'global_annotation_descriptions': struct {
        gnomad_sex_imputation_ploidy_cutoffs: struct {
            Description: str
        }, 
        gnomad_population_inference_pca_metrics: struct {
            Description: str
        }, 
        sample_hard_filter_cutoffs: struct {
            Description: str
        }, 
        gnomad_sample_qc_metric_outlier_cutoffs: struct {
            Description: str
        }, 
        gnomad_age_distribution: struct {
            Description: str, 
            sub_globals: struct {
                bin_edges: struct {
                    Description: str
                }, 
                bin_freq: struct {
                    Description: str
                }, 
                n_smaller: struct {
                    Description: str
                }, 
                n_larger: struct {
                    Description: str
                }
            }
        }, 
        hgdp_tgp

In [56]:
# One last validity check before writing out the dataset is to make sure we still have the number of rows we'd expect
# In this case, since the data is grouped by genetic region, population, the number of rows should be equal to the number of populations (78)
table.count()

2022-11-18 21:10:47 Hail: INFO: Ordering unsorted dataset with network shuffle


78

# 7. Export Final Table
<br>
<details><summary>For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 
<br>
<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.export"> More on  <i> export() </i></a></li>

</details>

[Back to Index](#Index)

- Write out the final table in tsv format 

```python3
table.export(final_table_path, header=True)
```

[Back to Index](#Index)